# క్లౌడ్‌లో డేటా సైన్స్: "Azure ML SDK" విధానం

## పరిచయం

ఈ నోట్‌బుక్‌లో, Azure ML SDK ఉపయోగించి మోడల్‌ను శిక్షణ ఇవ్వడం, డిప్లాయ్ చేయడం మరియు వినియోగించడం ఎలా చేయాలో నేర్చుకుంటాము.

ముందస్తు అవసరాలు:
1. మీరు ఒక Azure ML వర్క్‌స్పేస్ సృష్టించారు.
2. మీరు [హార్ట్ ఫెయిల్యూర్ డేటాసెట్](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) ను Azure ML లో లోడ్ చేశారు.
3. మీరు ఈ నోట్‌బుక్‌ను Azure ML స్టూడియోలో అప్లోడ్ చేశారు.

తదుపరి దశలు:

1. ఉన్న వర్క్‌స్పేస్‌లో ఒక ఎక్స్‌పెరిమెంట్ సృష్టించండి.
2. ఒక కంప్యూట్ క్లస్టర్ సృష్టించండి.
3. డేటాసెట్‌ను లోడ్ చేయండి.
4. AutoMLConfig ఉపయోగించి AutoML ను కాన్ఫిగర్ చేయండి.
5. AutoML ఎక్స్‌పెరిమెంట్‌ను నడపండి.
6. ఫలితాలను పరిశీలించి ఉత్తమ మోడల్‌ను పొందండి.
7. ఉత్తమ మోడల్‌ను రిజిస్టర్ చేయండి.
8. ఉత్తమ మోడల్‌ను డిప్లాయ్ చేయండి.
9. ఎండ్‌పాయింట్‌ను వినియోగించండి.

## Azure మెషీన్ లెర్నింగ్ SDK-స్పెసిఫిక్ ఇంపోర్ట్స్


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Initialize Workspace
నిర్వహించబడిన కాన్ఫిగరేషన్ నుండి వర్క్‌స్పేస్ ఆబ్జెక్ట్‌ను ప్రారంభించండి. .\config.json వద్ద కాన్ఫిగ్ ఫైల్ ఉన్నదని నిర్ధారించుకోండి.


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Azure ML ప్రయోగం సృష్టించండి

మనం ఇప్పుడే ప్రారంభించిన వర్క్‌స్పేస్‌లో 'aml-experiment' అనే ప్రయోగాన్ని సృష్టిద్దాం.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## కంప్యూట్ క్లస్టర్ సృష్టించండి
మీ AutoML రన్ కోసం మీరు ఒక [కంప్యూట్ టార్గెట్](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) సృష్టించాలి.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Data
మీరు డేటాసెట్‌ను Azure ML కు అప్‌లోడ్ చేసినట్లు మరియు కీ డేటాసెట్ పేరుతోనే ఉన్నదని నిర్ధారించుకోండి.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## ఆటోఎంఎల్ కాన్ఫిగరేషన్


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## ఆటోఎంఎల్ రన్


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

## ఉత్తమ మోడల్‌ను సేవ్ చేయండి


In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## ఉత్తమ మోడల్‌ను అమలు చేయండి

ఉత్తమ మోడల్‌ను అమలు చేయడానికి క్రింది కోడ్‌ను నడపండి. మీరు Azure ML పోర్టల్‌లో అమలులో ఉన్న స్థితిని చూడవచ్చు. ఈ దశ కొంత సమయం తీసుకోవచ్చు.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## ఎండ్‌పాయింట్‌ను వినియోగించండి
క్రింది ఇన్‌పుట్ నమూనాకు మీరు ఇన్‌పుట్‌లు జోడించవచ్చు.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. అసలు పత్రం దాని స్వదేశీ భాషలోనే అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం సిఫార్సు చేయబడుతుంది. ఈ అనువాదం వాడకంలో ఏర్పడిన ఏవైనా అపార్థాలు లేదా తప్పుదారితీసే అర్థాలు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
